In [1]:
import sys
import os
import numpy as np
import pandas as pd
import random
sys.path.append('../source')
from utils import *

In [56]:
def mask_filter(images: dict, range: tuple, no_keys: int = -1):
    for key, img in images.copy().items(): 
        if get_percent_coverage(img) < range[0] or get_percent_coverage(img) > range[1]:
            del images[key]
   
    keys = images.keys()
    
    if no_keys > 0 and no_keys < len(keys):
        keys = random.sample(sorted(images.keys()), no_keys)
        
    return {k: images[k] for k in keys}

def image_filter(images: dict, no_keys=-1):
    for key, img in images.copy().items(): 
        if get_percent_coverage(img) != 0:
            del images[key]
   
    keys = images.keys()
    
    if no_keys > 0 and no_keys < len(keys):
        keys = random.sample(sorted(images.keys()), no_keys)
        
    return {k: images[k] for k in keys}

def generate_lst_dataset(
    images: list, 
    masks: list, 
    patch_size: tuple, 
    save_path: str,
    skip_mask = False, 
    image_filter = None,
    mask_filter = None,
    limit=5000
    ):
    """Generates LST dataset by retiling images masks. Masks are named by originalname_r_c

    Args:
        images (list): hdf images to retile
        masks (list): hdf masks to retile
        patch_size (tuple): size of patches
        image_filter (function): function returning a filtered list of images
        mask_filter (function): function returning a filtered list of masks
        save_path (str): directory to save images and masked images
        limit (int, optional): set limit of images to save. Defaults to 5000.
    """
    collection = pd.DataFrame(columns=['sample', 'ground_truth'])
    
    lst_tiles = {}
    for image in images:
        basename = os.path.splitext(os.path.basename(image))[0]
        lst = get_lst_day(image)
        tiles_temp = retile_and_name(lst, basename, patch_size)
        if image_filter is not None:
            tiles_temp = image_filter(tiles_temp)
        lst_tiles = {**lst_tiles, **tiles_temp}
    print('# of LST tiles: {}'.format(len(lst_tiles)))
        
    assert len(os.listdir(save_path)) == 0, 'Save path not empty'
    
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    if not os.path.exists(os.path.join(save_path, 'ground_truth')):
        os.makedirs(os.path.join(save_path, 'ground_truth'))
    
    if skip_mask:
        for image_name, image in lst_tiles.items():
            image_filename = image_name + '.npy'
            np.save(os.path.join(save_path, 'ground_truth', image_filename), image)
            collection.loc[len(collection.index)] = [ 
                image_filename,
                image_filename,
            ]       

    else:
        if not os.path.exists(os.path.join(save_path, 'masked_images')):
            os.makedirs(os.path.join(save_path, 'masked_images'))
        if not os.path.exists(os.path.join(save_path, 'masks')):
            os.makedirs(os.path.join(save_path, 'masks'))
        
        mask_tiles = {}
        for mask in masks:
            basename = os.path.splitext(os.path.basename(image))[0]
            mask = get_cloud_mask(mask)
            tiles_temp = retile_and_name(mask, basename, patch_size)
            if mask_filter is not None:
                tiles_temp = mask_filter(tiles_temp)
            mask_tiles = {**mask_tiles, **tiles_temp}
        print('# of mask tiles: {}'.format(len(mask_tiles)))

                    
        assert len(mask_tiles) * len(lst_tiles) <= limit, 'Limit Exceeded ({} total images)'.format(len(mask_tiles) * len(lst_tiles))
        assert len(mask_tiles) > 0 and len(lst_tiles) > 0, 'Either masks or images are empty'   
    
        for image_name, image in lst_tiles.items():
            image_filename = image_name + '.npy'
            np.save(os.path.join(save_path, 'ground_truth', image_filename), image)
            for mask_name, mask in mask_tiles.items():
                if not os.path.exists(os.path.join(save_path, 'masks', mask_name + 'npy')):
                    np.save(os.path.join(save_path, 'masks', mask_name + 'npy'), mask)
                    
                masked_lst = np.multiply(image, mask)
                masked_lst_filename = mask_name + '_' + image_name + '.npy'
                # ADD ASSERTIONS HERE

                np.save(os.path.join(save_path, 'masked_images', masked_lst_filename), masked_lst)

                collection.loc[len(collection.index)] = [ 
                    masked_lst_filename,
                    image_filename,
                ]      
             
    collection.to_csv(os.path.join(save_path, 'collection.csv'), index=False)
    print('Sample pairs created: {}'.format(len(collection)))

In [57]:
data_path = '../../data/'
raw_path = os.path.join(data_path, 'LST_miniset', 'raw')
samples = [os.path.join(raw_path, x) for x in os.listdir(raw_path)[0:3]]
masks = [os.path.join(raw_path, x) for x in os.listdir(raw_path)[0:1]]

In [59]:
generate_lst_dataset(
    images=samples, 
    masks=masks, 
    patch_size=(32, 32), 
    image_filter=lambda x: image_filter(x, 10),
    mask_filter=lambda x: mask_filter(x, (0.01, 1), 10),
    save_path=os.path.join(data_path, 'test1'),
    skip_mask=False
)

# of LST tiles: 30
# of mask tiles: 10
Sample pair created: 300


In [42]:
coll = pd.read_csv(os.path.join(data_path, 'test', 'collection.csv'), index_col=False)
coll['sample'].equals(coll['ground_truth'])

for i in coll.iterrows():
    assert (np.load(os.path.join(data_path, 'test', 'ground_truth', i[1]['sample'])) == np.load(os.path.join(data_path, 'test', 'ground_truth', i[1]['ground_truth']))).all()